In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

#Preprocessing - Training and testing split
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

#Exploratory Data Analysis
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

#Model
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LassoCV, RidgeCV, ElasticNetCV 

#Evaluation: 
from sklearn.metrics import mean_squared_error, r2_score
# It looks like ir is not available any more!
# from sklearn.metrics import mean_absolute_percentage_error

In [38]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

## Loading Training/Testing Data:

In [39]:
train_url = 'https://raw.githubusercontent.com/mturner49/pylovers-final-project/dev/data/train.csv'
test_url = 'https://raw.githubusercontent.com/mturner49/pylovers-final-project/dev/data/test.csv'

train_df = pd.read_csv(train_url, low_memory=False, error_bad_lines=False, index_col='Id')
test_df = pd.read_csv(test_url, low_memory=False, error_bad_lines=False, index_col='Id')
train_df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [40]:
# This is for checking datatypes
# train_df.info()
# test_df.info()

## Data Preprocessing:

## Missing Data Imputation

In [41]:
def missing_cnt_percent(df):
    # create df that consists of columns and the number of missing values for each 
    data = df.isnull().sum().sort_values(ascending=False)
    missing_df = pd.DataFrame(data=data, columns=['missing_cnt'])

    # add column and fill it with the percentage of those missing values
    missing_df['percent_missing'] = missing_df.missing_cnt.apply(lambda x : '{:.2f}'.format(x/df.shape[0] * 100)) 
    missing_df = missing_df[missing_df.missing_cnt > 0]
    print(missing_df)

In [42]:
missing_cnt_percent(train_df)

              missing_cnt percent_missing
PoolQC               1453           99.52
MiscFeature          1406           96.30
Alley                1369           93.77
Fence                1179           80.75
FireplaceQu           690           47.26
LotFrontage           259           17.74
GarageType             81            5.55
GarageCond             81            5.55
GarageFinish           81            5.55
GarageQual             81            5.55
GarageYrBlt            81            5.55
BsmtFinType2           38            2.60
BsmtExposure           38            2.60
BsmtQual               37            2.53
BsmtCond               37            2.53
BsmtFinType1           37            2.53
MasVnrArea              8            0.55
MasVnrType              8            0.55
Electrical              1            0.07


In [43]:
# These columns has NA value that is different from null(NaN)!
train_df.update(train_df[['PoolQC']].fillna('NA'))
train_df.update(train_df[['MiscFeature']].fillna('NA'))
train_df.update(train_df[['Alley']].fillna('NA'))
train_df.update(train_df[['Fence']].fillna('NA'))
train_df.update(train_df[['FireplaceQu']].fillna('NA'))

In [44]:
# All 'GarageType','GarageYrBlt','GarageFinish','GarageCars','GarageQual','GarageCond' columns has 1379 not null values!
# So, I tested the theory that exactly same rows has null for these columns!
# All 81 do not have Garage! So, we can put zero/'NA' instead!
train_df.loc[:,['GarageCars','GarageArea','GarageType','GarageYrBlt','GarageFinish','GarageCars','GarageQual','GarageCond']][train_df['GarageType'].isnull()]


,GarageCars,GarageArea,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageQual,GarageCond
Id,,,,,,,,
40,0,0,NaN,NaN,NaN,0,NaN,NaN
49,0,0,NaN,NaN,NaN,0,NaN,NaN
79,0,0,NaN,NaN,NaN,0,NaN,NaN
89,0,0,NaN,NaN,NaN,0,NaN,NaN
90,0,0,NaN,NaN,NaN,0,NaN,NaN
...,...,...,...,...,...,...,...,...
1350,0,0,NaN,NaN,NaN,0,NaN,NaN
1408,0,0,NaN,NaN,NaN,0,NaN,NaN
1450,0,0,NaN,NaN,NaN,0,NaN,NaN


In [45]:
# All 81 do not have Garage! So, we can put zero/'NA' instead!
train_df.update(train_df[['GarageYrBlt','GarageCars']].fillna(0))
train_df.update(train_df[['GarageType','GarageFinish','GarageQual','GarageCond']].fillna('NA'))

In [46]:
# Same for Basement: 'BsmtFinType2','BsmtExposure','BsmtQual','BsmtCond','BsmtFinType1'
train_df.loc[:,['TotalBsmtSF','BsmtUnfSF','BsmtFinType2','BsmtExposure','BsmtQual','BsmtCond','BsmtFinType1']][train_df['BsmtCond'].isnull()]

,TotalBsmtSF,BsmtUnfSF,BsmtFinType2,BsmtExposure,BsmtQual,BsmtCond,BsmtFinType1
Id,,,,,,,
18,0,0,NaN,NaN,NaN,NaN,NaN
40,0,0,NaN,NaN,NaN,NaN,NaN
91,0,0,NaN,NaN,NaN,NaN,NaN
103,0,0,NaN,NaN,NaN,NaN,NaN
157,0,0,NaN,NaN,NaN,NaN,NaN
183,0,0,NaN,NaN,NaN,NaN,NaN
260,0,0,NaN,NaN,NaN,NaN,NaN
343,0,0,NaN,NaN,NaN,NaN,NaN
363,0,0,NaN,NaN,NaN,NaN,NaN


In [47]:
# All 37 do not have Garage! So, we can put 'NA' instead!
train_df.update(train_df[['BsmtQual','BsmtCond','BsmtFinType1']].fillna('NA'))

In [48]:
# Same for Basement: 'BsmtFinType2','BsmtExposure'
train_df.loc[:,['TotalBsmtSF','BsmtUnfSF','BsmtFinType2','BsmtExposure','BsmtQual','BsmtCond','BsmtFinType1']][train_df['BsmtExposure'].isnull()]

,TotalBsmtSF,BsmtUnfSF,BsmtFinType2,BsmtExposure,BsmtQual,BsmtCond,BsmtFinType1
Id,,,,,,,
18,0,0,NaN,NaN,NA,NA,NA
40,0,0,NaN,NaN,NA,NA,NA
91,0,0,NaN,NaN,NA,NA,NA
103,0,0,NaN,NaN,NA,NA,NA
157,0,0,NaN,NaN,NA,NA,NA
183,0,0,NaN,NaN,NA,NA,NA
260,0,0,NaN,NaN,NA,NA,NA
343,0,0,NaN,NaN,NA,NA,NA
363,0,0,NaN,NaN,NA,NA,NA


In [49]:
train_df.at[949,'BsmtExposure']='No'

In [50]:
train_df.loc[:,['TotalBsmtSF','BsmtUnfSF','BsmtFinSF2','BsmtFinType2','BsmtExposure','BsmtQual','BsmtCond','BsmtFinType1']][train_df['BsmtFinType2'].isnull()]

,TotalBsmtSF,BsmtUnfSF,BsmtFinSF2,BsmtFinType2,BsmtExposure,BsmtQual,BsmtCond,BsmtFinType1
Id,,,,,,,,
18,0,0,0,NaN,NaN,NA,NA,NA
40,0,0,0,NaN,NaN,NA,NA,NA
91,0,0,0,NaN,NaN,NA,NA,NA
103,0,0,0,NaN,NaN,NA,NA,NA
157,0,0,0,NaN,NaN,NA,NA,NA
183,0,0,0,NaN,NaN,NA,NA,NA
260,0,0,0,NaN,NaN,NA,NA,NA
333,3206,1603,479,NaN,No,Gd,TA,GLQ
343,0,0,0,NaN,NaN,NA,NA,NA


In [51]:
# I'm going with 'Average Living Quarters'
train_df.at[333,'BsmtFinType2']='ALQ'
train_df.update(train_df[['BsmtFinType2','BsmtExposure']].fillna('NA'))

In [52]:
train_df.update(train_df[['Electrical']].fillna('SBrkr'))

In [53]:
#I'm not professional but I think there is no Masonry veneer type for the Vinyl and Cement!
train_df.at[530,'MasVnrType']='Stone'

# train_df['Set_of_Numbers'] = train_df['Set_of_Numbers'].fillna(0)
train_df.update(train_df[['MasVnrArea']].fillna(0))
train_df.update(train_df[['MasVnrType']].fillna('None'))

In [54]:
train_df['LotFrontage'] = train_df['LotFrontage'].fillna((train_df['LotFrontage'].mean()))

In [19]:
missing_cnt_percent(train_df)

Empty DataFrame
Columns: [missing_cnt, percent_missing]
Index: []


## Convert categorical string to numeric

Categorical Variable Analysis for ordered categories

In [20]:
# LotShape
quality_ratings1 = {
    "IR3" : 1,
	"IR2" : 2,
	"IR1" : 3,
	"Reg" : 4
}
train_df["LotShape_Num"] = train_df.LotShape.map(quality_ratings1)
test_df["LotShape_Num"] = test_df.LotShape.map(quality_ratings1)

In [21]:
	# LandContour
quality_ratings2 = {
	"Low" : 1,
	"HLS" : 2,
	"Bnk" : 3,
	"Lvl" : 4
}
train_df["LandContour_Num"] = train_df.LandContour.map(quality_ratings2)
test_df["LandContour_Num"] = test_df.LandContour.map(quality_ratings2)

In [22]:
	# LandSlope
quality_ratings3 = {
	"Sev" : 1,
	"Mod" : 2,
	"Gtl" : 3
}
train_df["LandSlope_Num"] = train_df.LandSlope.map(quality_ratings3)
test_df["LandSlope_Num"] = test_df.LandSlope.map(quality_ratings3)

In [23]:
	# ExterQual, ExterCond, HeatingQC, KitchenQual
quality_ratings4 = {
	"Po" : 1,
	"Fa" : 2,
	"TA" : 3,
	"Gd" : 4,
	"Ex" : 5
}
train_df["ExterQual_Num"] = train_df.ExterQual.map(quality_ratings4)
train_df["ExterCond_Num"] = train_df.ExterCond.map(quality_ratings4)
train_df["HeatingQC_Num"] = train_df.HeatingQC.map(quality_ratings4)
train_df["KitchenQual_Num"] = train_df.KitchenQual.map(quality_ratings4)

test_df["ExterQual_Num"] = test_df.ExterQual.map(quality_ratings4)
test_df["ExterCond_Num"] = test_df.ExterCond.map(quality_ratings4)
test_df["HeatingQC_Num"] = test_df.HeatingQC.map(quality_ratings4)
test_df["KitchenQual_Num"] = test_df.KitchenQual.map(quality_ratings4)

In [24]:
	# BsmtQual, BsmtCond, GarageQual, GarageCond, FireplaceQu
quality_ratings5 = {
	"NA" : 1,
	"Po" : 2,
	"Fa" : 3,
	"TA" : 4,
	"Gd" : 5,
	"Ex" : 6
}
train_df["BsmtQual_Num"] = train_df.BsmtQual.map(quality_ratings5)
train_df["BsmtCond_Num"] = train_df.BsmtCond.map(quality_ratings5)
train_df["GarageQual_Num"] = train_df.GarageQual.map(quality_ratings5)
train_df["GarageCond_Num"] = train_df.GarageCond.map(quality_ratings5)
train_df["FireplaceQu_Num"] = train_df.FireplaceQu.map(quality_ratings5)

test_df["BsmtQual_Num"] = test_df.BsmtQual.map(quality_ratings5)
test_df["BsmtCond_Num"] = test_df.BsmtCond.map(quality_ratings5)
test_df["GarageQual_Num"] = test_df.GarageQual.map(quality_ratings5)
test_df["GarageCond_Num"] = test_df.GarageCond.map(quality_ratings5)
test_df["FireplaceQu_Num"] = test_df.FireplaceQu.map(quality_ratings5)

In [25]:
	# BsmtExposure
quality_ratings7 = {
	"NA" : 1,
	"No" : 2,
	"Mn" : 3,
	"Av" : 4,
	"Gd" : 5
}
train_df["BsmtExposure_Num"] = train_df.BsmtExposure.map(quality_ratings7)
test_df["BsmtExposure_Num"] = test_df.BsmtExposure.map(quality_ratings7)

In [26]:
	# BsmtFinType1, BsmtFinType2
quality_ratings8 = {
	"NA" : 1,
	"Unf" : 2,
	"LwQ" : 3,
	"Rec" : 4,
	"BLQ" : 5,
	"ALQ" : 6,
	"GLQ" : 7
}
train_df["BsmtFinType1_Num"] = train_df.BsmtFinType1.map(quality_ratings8)
train_df["BsmtFinType2_Num"] = train_df.BsmtFinType2.map(quality_ratings8)

test_df["BsmtFinType1_Num"] = test_df.BsmtFinType1.map(quality_ratings8)
test_df["BsmtFinType2_Num"] = test_df.BsmtFinType2.map(quality_ratings8)

In [27]:
	# Functional
quality_ratings9 = {
	"Sal" : 1,
	"Sev" : 2,
	"Maj2" : 3,
	"Maj1" : 4,
	"Mod" : 5,
	"Min2" : 6,
	"Min1" : 7,
	"Typ" : 8
}
train_df["Functional_Num"] = train_df.Functional.map(quality_ratings9)

test_df["Functional_Num"] = test_df.Functional.map(quality_ratings9)

In [28]:
	# GarageFinish
quality_ratings10 = {
	"NA" : 1,
	"Unf" : 2,
	"RFn" : 3,
	"Fin" : 4
}
train_df["GarageFinish_Num"] = train_df.GarageFinish.map(quality_ratings10)

test_df["GarageFinish_Num"] = test_df.GarageFinish.map(quality_ratings10)

In [29]:
	# CentralAir
quality_ratings11 = {
	"N" : 1,
	"Y" : 2
}
train_df["CentralAir_Num"] = train_df.CentralAir.map(quality_ratings11)

test_df["CentralAir_Num"] = test_df.CentralAir.map(quality_ratings11)

In [30]:
	# PavedDrive
quality_ratings12 = {
	"N" : 1,
	"P" : 2,
	"Y" : 3
}
train_df["PavedDrive_Num"] = train_df.PavedDrive.map(quality_ratings12)

test_df["PavedDrive_Num"] = test_df.PavedDrive.map(quality_ratings12)

In [31]:
	# PoolQC
quality_ratings6 = {
	"NA" : 1,
	"Fa" : 2,
	"TA" : 3,
	"Gd" : 4,
	"Ex" : 5
}
train_df["PoolQC_Num"] = train_df.PoolQC.map(quality_ratings6)
test_df["PoolQC_Num"] = test_df.PoolQC.map(quality_ratings6)

In [32]:
	# MSSubClass
quality_ratings7 = {
    20:"St46nNAS", 
    30:"St45nOl", 
    40:"StwFinAAA", 
    45:"HalfStUnfinAAA", 
    50:"HalfStFinAA",
    60:"St46nN", 
    70:"St45nOl", 
    75:"HalfStAA", 
    80:"SplorML", 
    85:"SplFoy", 
    90:"DupASnA", 
    120:"StPud46nN", 
    150:"HalfStPudAA", 
    160:"StPud46nNN", 
    180:"PudML", 190:"FamConv"
}
train_df["MSSubClass_Cat"] = train_df.MSSubClass.map(quality_ratings7)
test_df["MSSubClass_Cat"] = test_df.MSSubClass.map(quality_ratings7)

In [33]:
# drop the columns where we replaced with numbers
drop_cols = ['ExterQual', 'BsmtQual', 'HeatingQC', 'KitchenQual','GarageQual','LotShape','LandContour','LandSlope','ExterCond','BsmtCond','GarageCond','FireplaceQu','BsmtExposure','BsmtFinType1','BsmtFinType2','Functional','GarageFinish','CentralAir','PavedDrive','PoolQC','MSSubClass']

for col in drop_cols:
    train_df.drop(col, axis=1, inplace=True)
    test_df.drop(col, axis=1, inplace=True)

## Exploratory Data Analysis (EDA)

- Correlation 

In [34]:
corr_matrix = train_df.corr()
corr_matrix['SalePrice'] = round(corr_matrix['SalePrice'],4)
print_full(corr_matrix['SalePrice'].sort_values(ascending=False))

SalePrice           1.0000
OverallQual         0.7910
GrLivArea           0.7086
ExterQual_Num       0.6826
KitchenQual_Num     0.6596
GarageCars          0.6404
GarageArea          0.6234
TotalBsmtSF         0.6136
1stFlrSF            0.6059
BsmtQual_Num        0.5852
FullBath            0.5607
GarageFinish_Num    0.5492
TotRmsAbvGrd        0.5337
YearBuilt           0.5229
FireplaceQu_Num     0.5204
YearRemodAdd        0.5071
MasVnrArea          0.4726
Fireplaces          0.4669
HeatingQC_Num       0.4276
BsmtFinSF1          0.3864
BsmtExposure_Num    0.3750
LotFrontage         0.3349
WoodDeckSF          0.3244
2ndFlrSF            0.3193
OpenPorchSF         0.3159
BsmtFinType1_Num    0.3049
HalfBath            0.2841
GarageQual_Num      0.2738
LotArea             0.2638
GarageCond_Num      0.2632
GarageYrBlt         0.2614
CentralAir_Num      0.2513
PavedDrive_Num      0.2314
BsmtFullBath        0.2271
BsmtUnfSF           0.2145
BsmtCond_Num        0.2126
BedroomAbvGr        0.1682
P

In [35]:
corr_matrix[(corr_matrix.SalePrice<0.05) & (corr_matrix.SalePrice>-0.05)].SalePrice.sort_values(ascending=False)

MoSold              0.0464
3SsnPorch           0.0446
ExterCond_Num       0.0189
BsmtFinType2_Num   -0.0003
BsmtFinSF2         -0.0114
BsmtHalfBath       -0.0168
MiscVal            -0.0212
LowQualFinSF       -0.0256
YrSold             -0.0289
Name: SalePrice, dtype: float64

In [36]:
train_df['TotSF_Factor'] = train_df['1stFlrSF']+train_df['2ndFlrSF']+train_df['TotalBsmtSF']
train_df["AddedArea_Factor"] = train_df["GrLivArea"] + train_df["GarageArea"]
train_df["TotBath_Factor"] = train_df["FullBath"] + (train_df["HalfBath"]*0.5)
train_df['Overall_Factor'] = train_df['OverallQual'] * train_df['OverallCond'] * train_df['YearBuilt']
train_df['Yr_Factor'] = train_df['YrSold'] - np.maximum(train_df['YearBuilt'],train_df['YearRemodAdd'])
train_df['Pool_Factor'] = train_df['PoolArea'].apply(lambda x: 1 if x > 0 else 0)
train_df['OpenOR3SsnPorch_Factor'] = train_df["OpenPorchSF"].apply(lambda x: 1 if x > 0 else 0) + train_df["3SsnPorch"].apply(lambda x: 1 if x > 0 else 0) + train_df["WoodDeckSF"].apply(lambda x: 1 if x > 0 else 0)
train_df['Kitchen2Flr_Ratio'] = train_df['KitchenAbvGr'] / (train_df['1stFlrSF'].apply(lambda x: 1 if x > 0 else 0) + train_df['2ndFlrSF'].apply(lambda x: 1 if x > 0 else 0) + train_df['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0))

## Not high corr
# train_df['BathPerBedRoom_Factor'] = train_df['FullBath'] / (train_df['BedroomAbvGr']+0.01)
# train_df['BsmtFin2TotBsmtSF_Ratio'] = (train_df['BsmtFinSF1']+train_df['BsmtFinSF2']) / (train_df['TotalBsmtSF']+0.01)
# train_df["Age_factor"] = train_df["YearRemodAdd"] - train_df["YearBuilt"]
# train_df['Options_Factor'] =train_df['Porch_Factor'] + train_df['BathPerRoom_Factor'] + train_df['Pool_Factor']  + train_df["Bsmt_factor"]
# train_df['GarageCars2Area_Ratio'] = train_df['GarageCars'] / (train_df['GarageArea']+0.01)
# train_df['Porch_Factor'] = (train_df['OpenPorchSF'] + train_df['EnclosedPorch']+ train_df['3SsnPorch']+ train_df['ScreenPorch']+0.01)
# train_df['2ndFlr_Factor'] = train_df['2ndFlrSF'].apply(lambda x: 1 if x > 0 else 0)
# train_df["Bsmt_factor"] = train_df["TotalBsmtSF"].apply(lambda x: 1 if x > 0 else 0)
# train_df['TotGrLiv2Lot_Ratio'] = train_df['GrLivArea'] / (train_df['LotArea']+0.01)
# train_df['TotSF2Lot_Ratio'] = (train_df['1stFlrSF']+train_df['2ndFlrSF']+train_df['TotalBsmtSF']) / (train_df['LotArea']+0.01)
# train_df['TotBsmtBath2BsmtSF_Ratio'] = (train_df['BsmtFullBath'] + train_df['BsmtHalfBath'])/(train_df['TotalBsmtSF']+0.01)
# train_df['ClosePorch_Factor'] = train_df['ScreenPorch'] + train_df['EnclosedPorch']
# train_df['LowQualFin2TotSF_Ratio'] = train_df['LowQualFinSF'] / (train_df['1stFlrSF']+train_df['2ndFlrSF']+train_df['TotalBsmtSF']+0.01)

NameError: name 'np' is not defined

In [ ]:
test_df['TotSF_Factor'] = test_df['1stFlrSF']+test_df['2ndFlrSF']+test_df['TotalBsmtSF']
test_df["AddedArea_Factor"] = test_df["GrLivArea"] + test_df["GarageArea"]
test_df["TotBath_Factor"] = test_df["FullBath"] + (test_df["HalfBath"]*0.5)
test_df['Overall_Factor'] = test_df['OverallQual'] * test_df['OverallCond'] * test_df['YearBuilt']
test_df['Yr_Factor'] = test_df['YrSold'] - np.maximum(test_df['YearBuilt'],test_df['YearRemodAdd'])
test_df['Pool_Factor'] = test_df['PoolArea'].apply(lambda x: 1 if x > 0 else 0)
test_df['OpenOR3SsnPorch_Factor'] = test_df["OpenPorchSF"].apply(lambda x: 1 if x > 0 else 0) + test_df["3SsnPorch"].apply(lambda x: 1 if x > 0 else 0) + test_df["WoodDeckSF"].apply(lambda x: 1 if x > 0 else 0)
test_df['Kitchen2Flr_Ratio'] = test_df['KitchenAbvGr'] / (test_df['1stFlrSF'].apply(lambda x: 1 if x > 0 else 0) + test_df['2ndFlrSF'].apply(lambda x: 1 if x > 0 else 0) + test_df['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0))


In [ ]:
drop_cols = ['MoSold','3SsnPorch','ExterCond_Num','BsmtFinType2_Num','BsmtFinSF2','BsmtHalfBath','MiscVal','LowQualFinSF','YrSold','Street','Utilities','LandSlope_Num','PoolQC_Num','MiscFeature']
for col in drop_cols:
    train_df.drop(col, axis=1, inplace=True)
    test_df.drop(col, axis=1, inplace=True)

In [ ]:
corr_matrix = train_df.corr()
corr_matrix['SalePrice'] = round(corr_matrix['SalePrice'],4)
print_full(corr_matrix['SalePrice'].sort_values(ascending=False))

In [ ]:
corr_matrix[(corr_matrix.SalePrice<0.1) & (corr_matrix.SalePrice>-0.1)].SalePrice.sort_values(ascending=False)

In [ ]:
drop_cols = ['KitchenAbvGr','PoolArea','OpenPorchSF','LandContour_Num','EnclosedPorch','ScreenPorch','OverallCond','FullBath','HalfBath']
for col in drop_cols:
    train_df.drop(col, axis=1, inplace=True)
    test_df.drop(col, axis=1, inplace=True)

- Variance Inflation Factor

In [ ]:
num_features = [col for col in train_df.columns if train_df[col].dtypes != 'O' and col not in "Id"]
train_df_vif = train_df.loc[:,train_df.columns.isin(num_features)]

In [ ]:
X = add_constant(train_df_vif)
print_full(pd.Series([variance_inflation_factor(X.values, i)
          for i in range(X.shape[1])], index=X.columns))

In [ ]:
drop_cols = ['GrLivArea','2ndFlrSF']
for col in drop_cols:
    train_df.drop(col, axis=1, inplace=True)
    test_df.drop(col, axis=1, inplace=True)

In [ ]:
num_features = [col for col in train_df.columns if train_df[col].dtypes != 'O' and col not in "Id"]
train_df_vif = train_df.loc[:,train_df.columns.isin(num_features)]

In [ ]:
X = add_constant(train_df_vif)
print_full(pd.Series([variance_inflation_factor(X.values, i)
          for i in range(X.shape[1])], index=X.columns))

## Outliers Analysis:

In [ ]:
def outlier_thresholds(df, var):
    quartile1 = df[var].quantile(0.05)
    quartile3 = df[var].quantile(0.95)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

def has_outliers(df, num_col_names):
    var_names = []
    for col in num_col_names:
        low_limit, up_limit = outlier_thresholds(df, col)
        if df[(df[col] > up_limit) | (df[col] < low_limit)].any(axis=None):
            number_of_outliers = df[(df[col] > up_limit) | (df[col] < low_limit)].shape[0]
            var_names.append(col)
    return var_names

def replace_with_thresholds(df, var):
    low_limit, up_limit = outlier_thresholds(df, var)
    df.loc[(df[var] < low_limit), var] = low_limit
    df.loc[(df[var] > up_limit), var] = up_limit

In [ ]:
has_outliers(train_df, num_features)

In [ ]:
for col in num_features:
    replace_with_thresholds(train_df, col)

has_outliers(train_df, num_features)

## Categorical Variable Analysis for non-ordered categories

In [ ]:
train_df = pd.get_dummies(data = train_df, columns = ["SaleCondition", "MSSubClass_Cat", "MSZoning", "LotConfig",
        "Neighborhood", "Condition1", "Condition2", "BldgType", "HouseStyle", "RoofStyle", "RoofMatl", "Exterior1st", 
        "Exterior2nd", "MasVnrType", "Foundation", "Heating", "Electrical", "GarageType", "SaleType", "Fence", "Alley"] , drop_first=True)

In [ ]:
train_df.head()

In [ ]:
test_df = pd.get_dummies(data = test_df, columns = ["SaleCondition", "MSSubClass_Cat", "MSZoning", "LotConfig",
        "Neighborhood", "Condition1", "Condition2", "BldgType", "HouseStyle", "RoofStyle", "RoofMatl", "Exterior1st", 
        "Exterior2nd", "MasVnrType", "Foundation", "Heating", "Electrical", "GarageType", "SaleType", "Fence", "Alley"] , drop_first=True)

In [ ]:
train_cols = train_df.columns
test_cols = test_df.columns

train_not_test = train_cols.difference(test_cols)
train_not_test

In [ ]:
train_not_test = train_not_test.drop(['SalePrice'])
train_not_test

In [ ]:
test_df = pd.concat([test_df,pd.DataFrame(
    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],index=test_df.index, columns=train_not_test)], axis=1
)

In [ ]:
train_cols = train_df.columns
test_cols = test_df.columns

test_not_train = test_cols.difference(train_cols)
test_not_train

In [ ]:
train_df = pd.concat([train_df,pd.DataFrame([[0]],index=test_df.index, columns=test_not_train)], axis=1)

## Split data set into train dataset and test dataset 

In [ ]:
#Independent variable:
x= train_df.loc[:,~train_df.columns.isin(['Id','SalePrice'])]

#dependent variable:
y=train_df.loc[:,'SalePrice']

In [ ]:
x_train, x_test, y_train, y_test= train_test_split(x,y,test_size=0.3,random_state=42)

# Standardization 

In [ ]:
#Independent variable:
X= train_df.loc[:,~train_df.columns.isin(['Id','SalePrice'])]

#dependent variable:
Y=train_df[['SalePrice']]

In [ ]:
sc_X = StandardScaler()
sc_Y = StandardScaler()

In [ ]:
X_s = sc_X.fit_transform(X)
Y_s = sc_X.fit_transform(Y)

In [ ]:
x_s_train, x_s_test, y_s_train, y_s_test= train_test_split(X_s,Y_s,test_size=0.3,random_state=42)

# Model

In [ ]:
def percentage_error(actual, predicted):
    res = np.empty(actual.shape)
    for j in range(actual.shape[0]):
        if actual[j] != 0:
            res[j] = (actual[j] - predicted[j]) / actual[j]
        else:
            res[j] = predicted[j] / np.mean(actual)
    return res

def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs(percentage_error(np.asarray(y_true), np.asarray(y_pred)))) * 100

## Linear Regression

In [ ]:
ln_reg=LinearRegression()

In [ ]:
ln_reg.fit(x_train,y_train)

In [ ]:
ln_y_pred=ln_reg.predict(x_test)

In [ ]:
print('MAPE = ', mean_absolute_percentage_error(y_test,ln_y_pred),
      '\nMSE = ', mean_squared_error(y_test,dt_y_pred),
      '\nr2_score = ', r2_score(y_test,ln_y_pred))

## Decision Tree

In [ ]:
regressor_dt = DecisionTreeRegressor()
regressor_dt.fit(x_train, y_train)

In [ ]:
dt_y_pred=regressor_dt.predict(x_test)

In [ ]:
# mean_absolute_percentage_error(y_test,dt_y_pred)
print('MAPE = ', mean_absolute_percentage_error(y_test,dt_y_pred), 
      '\nMSE = ', mean_squared_error(y_test,dt_y_pred),
      '\nr2_score = ', r2_score(y_test,dt_y_pred))

- With Scaled Data

In [ ]:
regressor_dt = DecisionTreeRegressor()
regressor_dt.fit(x_s_train, y_s_train)

In [ ]:
dts_y_pred=regressor_dt.predict(x_s_test)

In [ ]:
mean_absolute_percentage_error(y_s_test,dts_y_pred)
print('MAPE = ', mean_absolute_percentage_error(y_test,dts_y_pred), 
      '\nMSE = ', mean_squared_error(y_test,dts_y_pred),
      '\nr2_score = ', r2_score(y_test,dts_y_pred))

## SVR

In [ ]:
svr_regressor = SVR(kernel = 'rbf')

In [ ]:
svr_regressor.fit(x_train,y_train)

In [ ]:
svr_y_pred = svr_regressor.predict(x_test)

In [ ]:
print('MAPE = ', mean_absolute_percentage_error(y_test,svr_y_pred), 
      '\nMSE = ', mean_squared_error(y_test,svr_y_pred),
      '\nr2_score = ', r2_score(y_test,svr_y_pred))


- kernel = rbf


In [ ]:
svr_rbf = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)
svr_rbf.fit(x_train,y_train)

In [ ]:
svr_rbf_y_pred = svr_rbf.predict(x_test)

In [ ]:
mean_absolute_percentage_error(y_test,svr_rbf_y_pred)
print('MAPE = ', mean_absolute_percentage_error(y_test,svr_rbf_y_pred), 
      '\nMSE = ', mean_squared_error(y_test,svr_rbf_y_pred),
      '\nr2_score = ', r2_score(y_test,svr_rbf_y_pred))

- kernel = linear

In [ ]:
svr_lin = SVR(kernel='linear', C=100, gamma='auto')
svr_lin.fit(x_train,y_train)

In [ ]:
svr_lin_y_pred = svr_lin.predict(x_test)

In [ ]:
print('MAPE = ', mean_absolute_percentage_error(y_test,svr_lin_y_pred), 
      '\nMSE = ', mean_squared_error(y_test,svr_lin_y_pred),
      '\nr2_score = ', r2_score(y_test,svr_lin_y_pred))

- kernel = poly

In [ ]:
# It takes a long time to run this model!
# svr_poly = SVR(kernel='poly', C=100, gamma='auto', degree=3, epsilon=.1,
#                coef0=1)
# svr_poly.fit(x_train,y_train)

In [ ]:
# svr_poly_y_pred = svr_poly.predict(x_test)

In [ ]:
# print('MAPE = ', mean_absolute_percentage_error(y_test,svr_poly_y_pred), 
#       '\nMSE = ', mean_squared_error(y_test,svr_poly_y_pred),
#       '\nr2_score = ', r2_score(y_test,svr_poly_y_pred))

## Random Forest

In [ ]:
rf_regressor = RandomForestRegressor()
rf_regressor.fit(x_train,y_train)

In [ ]:
rf_y_pred=rf_regressor.predict(x_test)

In [ ]:
print('MAPE = ', mean_absolute_percentage_error(y_test,rf_y_pred), 
      '\nMSE = ', mean_squared_error(y_test,rf_y_pred),
      '\nr2_score = ', r2_score(y_test,rf_y_pred))

- With Scaled Data

In [ ]:
rf_regressor = RandomForestRegressor()
rf_regressor.fit(x_s_train,y_s_train)

In [ ]:
rf_s_y_pred=rf_regressor.predict(x_s_test)

In [ ]:
print('MAPE = ', mean_absolute_percentage_error(y_test,rf_s_y_pred), 
      '\nMSE = ', mean_squared_error(y_test,rf_s_y_pred),
      '\nr2_score = ', r2_score(y_test,rf_s_y_pred))

## LASSO regression

In [ ]:
lasso_regressor = LassoCV()
lasso_regressor.fit(x_train,y_train)

In [ ]:
lasso_y_pred=lasso_regressor.predict(x_test)

In [ ]:
print('MAPE = ', mean_absolute_percentage_error(y_test,lasso_y_pred), 
      '\nMSE = ', mean_squared_error(y_test,lasso_y_pred),
      '\nr2_score = ', r2_score(y_test,lasso_y_pred))

## Ridge Regression

In [ ]:
ridge_regressor = RidgeCV()
ridge_regressor.fit(x_train,y_train)

In [ ]:
ridge_y_pred=ridge_regressor.predict(x_test)

In [ ]:
print('MAPE = ', mean_absolute_percentage_error(y_test,ridge_y_pred), 
      '\nMSE = ', mean_squared_error(y_test,ridge_y_pred),
      '\nr2_score = ', r2_score(y_test,ridge_y_pred))

## ElasticNet Regression

In [ ]:
el_regressor = ElasticNetCV()
el_regressor.fit(x_train,y_train)

In [ ]:
el_y_pred=el_regressor.predict(x_test)

In [ ]:
print('MAPE = ', mean_absolute_percentage_error(y_test,el_y_pred), 
      '\nMSE = ', mean_squared_error(y_test,el_y_pred),
      '\nr2_score = ', r2_score(y_test,el_y_pred))

## Data Exploration:

## Data Visualization (CAN BE DELETED):